In [2]:
client_id = 'cb091220adca45a0ad6e7260ceb53e8d'
client_secret = 'ff0053cbd1b841cd96bd858aca617158'

In [3]:
import base64
import six
import requests    

In [4]:
#creates query header(spotify auth token)

def _make_authorization_headers(client_id, client_secret):
    auth_header = base64.b64encode(
        six.text_type(client_id + ":" + client_secret).encode("ascii")
    )
    return {"Authorization": "Basic %s" % auth_header.decode("ascii")}

headers = _make_authorization_headers(client_id, client_secret)
payload = {"grant_type": "client_credentials"}
OAUTH_TOKEN_URL = "https://accounts.spotify.com/api/token"
response = requests.post(
    OAUTH_TOKEN_URL,
    data=payload,
    headers=headers
)

def make_bearer_headers(access_token):
    return {"Authorization": "Bearer %s" % access_token}

query_header = make_bearer_headers(response.json()['access_token'])
query_header

In [24]:
# function to retrieve URI given the track and artist name
def getURI(track, artist):
    SEARCH_URL = "https://api.spotify.com/v1/search"
    params = {
        'q': f"track:{track} artist:{artist}",
        'type': "track"
    }
    response = requests.get(SEARCH_URL, params = params, headers = query_header)
    testresult = response.json()['tracks']['items']
    URI= response.json()['tracks']['items'][0]['uri']
    return URI


In [25]:
#testing getURI function
track = 'Without Me'
artist = 'Halsey'

getURI(track, artist)

'spotify:track:6FZDfxM3a3UCqtzo5pxSLZ'

In [21]:
# copy csv, de-duplicate tracks, clean track/artist titles by replacing weird stuff with spaces, split into two columns, drop "single" column, apply getURI to each row

'spotify:track:3e9HZxeyfWwjeyPAMmWSSQ'